## this script use fixed-effect model to estimate the importance of different funding sources

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import tqdm

In [2]:
pub_path = "../../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"

In [3]:
df = pd.read_pickle(pub_path)

In [4]:
df.sample(2)

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
2334018,51212588,2013,82,1,[Iran],Not-Funded,[Iran],Not-Funded,0,0,0,0
6821579,47688318,2011,8,1,[Greece],Not-Funded,[Greece],Not-Funded,0,0,0,0


In [5]:
def get_pubcnt_cntry_year(df):
    df = df[['id','year','author_distinct']].explode('author_distinct')
    df = df.groupby(['author_distinct','year']).size().reset_index(name='pubcnt')
    df = df.rename(columns={'author_distinct':'cntry'})
    return df

In [6]:
def get_fundcnt_from_cntrys(df):
    df = df[df.funder!='Not-Funded']
    pub2cntry = df[['id','year','author_distinct','funder']].explode('author_distinct') #assign publications into author countries
    fund2cntry = pub2cntry.explode('funder').groupby(['author_distinct','year','funder']).size().reset_index(name='fundcnt')
    fund2cntry = fund2cntry.rename(columns={'author_distinct':'cntry'})
    return fund2cntry

In [7]:
def map_funder_to_new_name(fund2cntry, topcntrys):
    topcntrys_dict = dict(zip(topcntrys, topcntrys))
    #replace the non-top funders to "others"
    df = fund2cntry.copy()
    df['region'] = df['funder'].copy()
    df['region'] = df['region'].map(topcntrys_dict)
    df['region'] = df['region'].fillna('others')
    df.loc[df['cntry']==df['funder'],'region']='domestic' #assign "domestic"
    return df

In [8]:
def group_fund_by_region(fund2cntry):
    #group the amount of fund by region
    fund_by_region = fund2cntry.groupby(['cntry','year','region'])['fundcnt'].sum().reset_index() 
    fund_by_region = fund_by_region.pivot_table(
        index=['cntry','year'], columns=['region'],values='fundcnt',fill_value=0)
    fund_by_region['foreign'] = fund_by_region.loc[:, fund_by_region.columns != 'domestic'].sum(axis=1)
    #for those regions that 
    fund_by_region = np.log(fund_by_region+1).reset_index()
    return fund_by_region

In [9]:
def add_previous_pub(pubcnt):
    df_curr = pubcnt[pubcnt.year>2009]
    df_curr = df_curr.rename(columns={'year':'year_curr','pubcnt':'pubcnt_curr'})
    df_prev = pubcnt[pubcnt.year<2018]
    df_prev = df_prev.rename(columns={'year':'year_prev','pubcnt':'pubcnt_prev'})
    df_curr['year_prev'] = df_curr['year_curr']-1
    df = df_curr.merge(df_prev, on=['cntry','year_prev'])
    return df

#### get the amount of grants from each top countries

In [10]:
fund_by_cntry = get_fundcnt_from_cntrys(df)
topcntrys=['United States','China','EU','United Kingdom','France','Germany']
fund_by_cntry_region = map_funder_to_new_name(fund_by_cntry, topcntrys)
fund_by_region = group_fund_by_region(fund_by_cntry_region)

In [11]:
test = fund_by_cntry_region.copy()

In [31]:
test[(test.cntry=='Australia')&(test.year==2009)].sort_values(by='fundcnt',ascending=False)

21233

In [45]:
test[(test.year==2009)].sort_values(by='fundcnt',ascending=False).head(60)

,cntry,year,funder,fundcnt,region
14365,China,2009,China,342831,domestic
78986,United States,2009,United States,312582,domestic
26928,Germany,2009,Germany,59155,domestic
37396,Japan,2009,Japan,55454,domestic
67465,Spain,2009,Spain,52987,domestic
12271,Canada,2009,Canada,41853,domestic
77635,United Kingdom,2009,United Kingdom,39148,domestic
8400,Brazil,2009,Brazil,33299,domestic
24286,France,2009,France,30869,domestic
66618,South Korea,2009,South Korea,30623,domestic


In [13]:
df.head(2)

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,60694041,2017,9,1,"[Canada, United States]","[United States, United States]","[Canada, United States]",[United States],1,1,0,0
1,48573156,2012,89,1,[France],Not-Funded,[France],Not-Funded,0,0,0,0


In [33]:
tt1 = df[(df.year==2009)&(df.funder!='Not-Funded')].explode("author_distinct")

In [34]:
t = tt1[tt1.author_distinct=='Australia']
t

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
1236,31887308,2009,100,1,"[Japan, Australia]",[Japan],Australia,[Japan],1,1,0,0
1694,31764816,2009,51,1,"[Australia, United States, Germany]","[United States, United States, United States, ...",Australia,"[United States, Germany]",1,1,1,1
2339,32220618,2009,37,1,"[Australia, Singapore, Greece, Israel, Italy]","[United Kingdom, Israel, Australia, EU]",Australia,"[Israel, EU, Australia, United Kingdom]",1,1,1,1
2656,31256583,2009,12,1,"[United Kingdom, China, South Korea, Japan, Au...",[South Korea],Australia,[South Korea],1,1,0,0
4397,31416613,2009,51,1,"[United States, Taiwan, Australia]","[Taiwan, Taiwan, Taiwan]",Australia,[Taiwan],1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12752899,32817862,2009,98,1,"[China, Australia]",[Finland],Australia,[Finland],1,1,0,1
12753108,32854986,2009,28,1,"[Australia, United States, Czech Republic]","[United States, Australia, Australia, Czech Re...",Australia,"[Czech Republic, United States, Australia]",1,1,1,1
12755487,32086536,2009,17,1,"[Australia, Ireland, New Zealand, United Kingdom]","[Australia, Australia, Australia, Australia]",Australia,[Australia],1,1,0,0
12756790,32847934,2009,99,1,[Australia],[Australia],Australia,[Australia],1,0,0,0


In [37]:
t['t'] = t.apply(lambda x:1 if len(x.funder_distinct)==1 and x.funder_distinct[0]=='Australia' else 0, axis=1)

/tmp/ipykernel_2156404/3051843278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['t'] = t.apply(lambda x:1 if len(x.funder_distinct)==1 and x.funder_distinct[0]=='Australia' else 0, axis=1)


In [42]:
t[t.t==0].explode('funder_distinct').groupby('funder_distinct').size().sort_values(ascending=False)

funder_distinct
Australia         3173
United States     2900
China             1164
United Kingdom    1152
Germany            531
                  ... 
Brunei               1
Bulgaria             1
Nigeria              1
Iraq                 1
Zimbabwe             1
Length: 89, dtype: int64

In [40]:
8270/15956

0.5183003258962146

#### get the number of publications authored by countries between two consecutive years

In [49]:
pubcnt = get_pubcnt_cntry_year(df)
pubcnt_new = add_previous_pub(pubcnt)

In [50]:
def calculate_growth_rate_take_log(df):
    df['growth_rate']=(df['pubcnt_curr']/df['pubcnt_prev'])-1
    #df['growth_rate']=df['growth_rate']
    df['pubcnt_curr']=np.log(df['pubcnt_curr'])
    df['pubcnt_prev']=np.log(df['pubcnt_prev'])
    return df

In [51]:
pubcnt_new=calculate_growth_rate_take_log(pubcnt_new)

In [52]:
pubcnt_new.head(2)

,cntry,year_curr,pubcnt_curr,year_prev,pubcnt_prev,growth_rate
0,Afghanistan,2010,3.496508,2009,2.833213,0.941176
1,Afghanistan,2011,3.332205,2010,3.496508,-0.151515


In [53]:
reg_df = pubcnt_new.merge(fund_by_region, left_on=['cntry','year_curr'], right_on=['cntry','year']
                         ).drop(columns=['year'])

In [56]:
reg_df.to_csv("regression_data.csv", index=False)

In [25]:
reg_df = pd.read_csv("regression_data.csv")

In [26]:
reg_df['test'] = reg_df.apply(lambda x: 1 if x.foreign>x.domestic else 0, axis=1)

In [30]:
reg_df[reg_df.test==1].cntry.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antarctica', 'Antigua & Barbuda', 'Argentina', 'Armenia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belgium', 'Belize', 'Benin', 'Bermuda',
       'Bhutan', 'Bolivia', 'Bosnia & Herzegovina', 'Botswana', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Byelarus',
       'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Central African Republic', 'Chad', 'Chile', 'Colombia', 'Comoros',
       'Congo', 'Cook Islands', 'Costa Rica', 'Crimea', 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica ',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia',
       'Federated States of Micronesia ', 'Fiji', 'Finland', 'France',
       'French-Guiana', 'French-Polynesia', 'Gabon', 'Gambia', 'Georgia',
       'Germany', 'Ghana ', 'Greece', 'G

In [64]:
reg_df.shape

(1782, 16)